<a href="https://colab.research.google.com/github/SabrineRiahi/Key_Localisation_FloydHub/blob/master/Darknet21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Darknet

In [0]:
!git clone https://github.com/pjreddie/darknet
%cd ./darknet
!make

# Training yolo on VOC


## Downloading datas

In [0]:
!wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
!wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!wget https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
!tar xf VOCtrainval_11-May-2012.tar
!tar xf VOCtrainval_06-Nov-2007.tar
!tar xf VOCtest_06-Nov-2007.tar

In [11]:
# Generate labels
%cd /content/darknet/data
!wget https://pjreddie.com/media/files/voc_label.py
!python voc_label.py

/content/darknet/data
--2019-11-09 12:24:17--  https://pjreddie.com/media/files/voc_label.py
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2042 (2.0K) [application/octet-stream]
Saving to: ‘voc_label.py’

voc_label.py        100%[===================>]   1.99K  --.-KB/s    in 0s      

2019-11-09 12:24:18 (459 MB/s) - ‘voc_label.py’ saved [2042/2042]



## Adding key's data 

In [0]:
# Adding key's images
%cd /content
!git clone https://github.com/belarbi2733/keras_yolov3

%cd /content/keras_yolov3
!wget https://github.com/belarbi2733/keras_yolov3/releases/download/1/key_wb.zip
!wget https://github.com/belarbi2733/keras_yolov3/releases/download/1/bckgrnd.zip
!unzip bckgrnd.zip
!unzip key_wb.zip
!python keys_with_background.py --keys "key_wb" --background "bckgrnd" --output "/content/darknet/data/VOCdevkit/VOC2007/JPEGImages"

In [0]:
# Rename annotations.csv file into txt file
%cd /content/darknet/data/VOCdevkit/VOC2007/JPEGImages
pre, ext = os.path.splitext(annotations.csv)
os.rename(annotations.csv, pre + txt)
!mv annotations.txt /content/darknet/data/VOCdevkit/VOC2007/labels

In [0]:
# Generate 1 file "annotation.txt" for each image in the right directory
%cd /content/darknet/data/VOCdevkit/VOC2007/labels
!python annotation_pythonv3.py

In [0]:
# Generate annotation_train.txt and annotation_test.txt == Directories of file img of keys
%cd /content/darknet/data/VOCdevkit/VOC2007/labels
!python annotation_train_test.py

In [0]:
# Concatenate annotation_train.txt + 2007_train.txt = train.txt
filenames = ['annotation_train.txt', '2007_train.txt']
with open('/content/darknet/data/train.txt', 'w') as outfile : # Path to output file
    for fname in filenames :
        with open(fname) as infile :
            for line in infile :
                outfile.write(line)
# Concatenate annotation_test.txt + 2007_test.txt = test.txt
filenames = ['annotation_test.txt', '2007_test.txt']
with open('/content/darknet/data/test.txt', 'w') as outfile : # Path to output file
    for fname in filenames :
        with open(fname) as infile :
            for line in infile :
                outfile.write(line)
!mv train.txt /content/darknet/data
!mv test.txt /content/darknet/data

## Modifying files .voc .names .cfg

In [0]:
# modifier .cfg : nb classes, nb filtes
# modifier .names : 81 = clés
# modifier .voc : paths
# modifier .txt : path of img data test/train

## Downloading pretrained convolutionnal  weights

In [0]:
%cd darknet
!wget https://pjreddie.com/media/files/darknet53.conv.74

## Training model

In [0]:
!./darknet convert.py detector train cfg/voc.data cfg/yolov3-voc.cfg darknet53.conv.74

## Testing model

In [0]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

%cd /darknet
#!wget https://pjreddie.com/media/files/yolov3.weights
!./darknet detect cfg/yolov3.cfg yolov3.weights data/kite.jpg
imShow('predictions.jpg')